# Classification with Gated Residual and Variable Selection Networks

**Author:** [Mike Fournigault](https://www.linkedin.com/in/mike-fournigault-57312071/)<br>
**Description:** Using Gated Residual and Variable Selection Networks for quality assessment of astronomical images.
Freely inspired by the work of [Khalid Salama](https://keras.io/examples/structured_data/classification_with_grn_and_vsn/)

## Introduction

This example demonstrates the use of Gated
Residual Networks (GRN) and Variable Selection Networks (VSN), proposed by
Bryan Lim et al. in
[Temporal Fusion Transformers (TFT) for Interpretable Multi-horizon Time Series Forecasting](https://arxiv.org/abs/1912.09363),
for structured data classification. GRNs give the flexibility to the model to apply
non-linear processing only where needed. VSNs allow the model to softly remove any
unnecessary noisy inputs which could negatively impact performance.
Together, those techniques help improving the learning capacity of deep neural
network models.

Note that this example implements only the GRN and VSN components described in
in the paper, rather than the whole TFT model, as GRN and VSN can be useful on
their own for structured data learning tasks.


To run the code you need to use TensorFlow 2.3 or higher.

In [5]:
import os
import sys

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers

sys.path.append(os.path.abspath("../../src/"))
import mapping_fits_objects as mfo

## Loading and preparing the datasets

Reading and merging catalog and mapping files

In [6]:
columns = ["OBJECT_ID", "FITS_ID", "CCD_ID", "ISO0", "BACKGROUND", "ELLIPTICITY", "ELONGATION", "CLASS_STAR", "FLAGS", "EXPTIME"]
data_path = "../../data/"
proc_path = os.path.join(data_path, "processed")
fm_path = os.path.join(data_path, "for_modeling")


In [7]:
cdc = pd.read_parquet(os.path.join(fm_path, "objects_catalog_ngc7000.parquet.gz"), engine='auto')

In [8]:
# Read all catalogs

# catalog_cadc = mfo.read_catalog(os.path.join(fm_path, "som_objects_catalog_cadc.hdf5"), columns)
# catalog_ngc0869 = mfo.read_catalog(os.path.join(fm_path, "som_objects_catalog_ngc0869.hdf5"), columns)
# catalog_ngc0896 = mfo.read_catalog(os.path.join(fm_path, "som_objects_catalog_ngc0896.hdf5"), columns)
# catalog_ngc7000 = mfo.read_catalog(os.path.join(fm_path, "som_objects_catalog_ngc7000.hdf5"), columns)
catalog_cadc = pd.read_parquet(os.path.join(fm_path, "objects_catalog_cadc2.parquet.gz"), engine='auto')
catalog_ngc0869 = pd.read_parquet(os.path.join(fm_path, "objects_catalog_ngc0869.parquet.gz"), engine='auto')
catalog_ngc0896 = pd.read_parquet(os.path.join(fm_path, "objects_catalog_ngc0896.parquet.gz"), engine='auto')
catalog_ngc7000 = pd.read_parquet(os.path.join(fm_path, "objects_catalog_ngc7000.parquet.gz"), engine='auto')


# # Read all mapping files
# mapping_fits_obj_cadc = pd.read_parquet(os.path.join(fm_path, "fits_objects_mapping_cadc2.parquet.gz"), engine='auto')
# mapping_fits_obj_ngc0869 = pd.read_parquet(os.path.join(fm_path, "fits_objects_mapping_ngc0869.parquet.gz"), engine='auto')
# mapping_fits_obj_ngc0896 = pd.read_parquet(os.path.join(fm_path, "fits_objects_mapping_ngc0896.parquet.gz"), engine='auto')
# mapping_fits_obj_ngc7000 = pd.read_parquet(os.path.join(fm_path, "fits_objects_mapping_ngc7000.parquet.gz"), engine='auto')

# Concatenate dataframes
catalog = pd.concat([catalog_cadc, catalog_ngc0869, catalog_ngc0896, catalog_ngc7000], ignore_index=True)
# mapping = pd.concat([mapping_fits_obj_cadc, mapping_fits_obj_ngc0869, mapping_fits_obj_ngc0896, mapping_fits_obj_ngc7000], ignore_index=True)



Creating the train/val/test split

In [11]:
# Create a selection mask for the training / validation / test set
train_selec = np.random.rand(len(catalog.index)) < 0.8
val_test_selec = ~train_selec
val_selec = np.random.rand(len(val_test_selec)) < 0.5
test_selec = ~val_selec

# Create the training, validation and test sets
# train_mapping = mapping[train_selec]
# val_mapping = mapping[val_selec]
# test_mapping = mapping[test_selec]

catalog_train = catalog[train_selec]
catalog_val = catalog[val_selec]
catalog_test = catalog[test_selec]

Saving the splitted datasets to files.

In [12]:
# Save the training, validation and test sets
# train_mapping.to_parquet(os.path.join(fm_path, "train_mapping.parquet.gz"), compression='gzip')
# val_mapping.to_parquet(os.path.join(fm_path, "val_mapping.parquet.gz"), compression='gzip')
# test_mapping.to_parquet(os.path.join(fm_path, "test_mapping.parquet.gz"), compression='gzip')

catalog_train.to_parquet(os.path.join(fm_path, "train_catalog.parquet.gz"), compression='gzip')
catalog_val.to_parquet(os.path.join(fm_path, "val_catalog.parquet.gz"), compression='gzip')
catalog_test.to_parquet(os.path.join(fm_path, "test_catalog.parquet.gz"), compression='gzip')

## Define dataset metadata

Here, we define the metadata of the dataset that will be useful for reading and
parsing the data into input features, and encoding the input features with respect
to their types.

In [14]:
# Target feature name.
TARGET_FEATURE_NAME = "gt_label1"
# Weight column name.
WEIGHT_COLUMN_NAME = "instance_weight"
# Numeric feature names.
NUMERIC_FEATURE_NAMES = [
    "ISO0", 
    "BACKGROUND", 
    "ELLIPTICITY", 
    "ELONGATION", 
    "CLASS_STAR", 
    "FLAGS", 
    "EXPTIME"
]
# Categorical features and their vocabulary lists.
# Note that we add 'v=' as a prefix to all categorical feature values to make
# sure that they are treated as strings.
# CATEGORICAL_FEATURES_WITH_VOCABULARY = {
#     feature_name: sorted([str(value) for value in list(data[feature_name].unique())])
#     for feature_name in CSV_HEADER
#     if feature_name
#     not in list(NUMERIC_FEATURE_NAMES + [WEIGHT_COLUMN_NAME, TARGET_FEATURE_NAME])
# }
CATEGORICAL_FEATURES_WITH_VOCABULARY = {}
# All features names.
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + list(
    CATEGORICAL_FEATURES_WITH_VOCABULARY.keys()
)
# Feature default values.
COLUMN_DEFAULTS = [
    [0.0]
    if feature_name in NUMERIC_FEATURE_NAMES + [TARGET_FEATURE_NAME, WEIGHT_COLUMN_NAME]
    else ["NA"]
    for feature_name in FEATURE_NAMES
]

## Create a `tf.data.Dataset` for training and evaluation

We create an input function to read and parse the file, and convert features and
labels into a [`tf.data.Dataset`](https://www.tensorflow.org/guide/datasets) for
training and evaluation.

In [15]:

def process(features, target):
    for feature_name in features:
        if feature_name in CATEGORICAL_FEATURES_WITH_VOCABULARY:
            # Cast categorical feature values to string.
            features[feature_name] = keras.ops.cast(features[feature_name], "string")
    # Get the instance weight.
    weight = features.pop(WEIGHT_COLUMN_NAME)
    return features, target, weight


def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=False,
        shuffle=shuffle,
    ).map(process)

    return dataset


## Create model inputs

In [16]:

def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="float32"
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="string"
            )
    return inputs


## Encode input features

For categorical features, we encode them using `layers.Embedding` using the
`encoding_size` as the embedding dimensions. For the numerical features,
we apply linear transformation using `layers.Dense` to project each feature into
`encoding_size`-dimensional vector. Thus, all the encoded features will have the
same dimensionality.

In [17]:

def encode_inputs(inputs, encoding_size):
    encoded_features = []
    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURES_WITH_VOCABULARY:
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            # Create a lookup to convert a string values to an integer indices.
            # Since we are not using a mask token nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and  num_oov_indices to 0.
            index = layers.StringLookup(
                vocabulary=vocabulary, mask_token=None, num_oov_indices=0
            )
            # Convert the string input values into integer indices.
            value_index = index(inputs[feature_name])
            # Create an embedding layer with the specified dimensions
            embedding_ecoder = layers.Embedding(
                input_dim=len(vocabulary), output_dim=encoding_size
            )
            # Convert the index values to embedding representations.
            encoded_feature = embedding_ecoder(value_index)
        else:
            # Project the numeric feature to encoding_size using linear transformation.
            encoded_feature = keras.ops.expand_dims(inputs[feature_name], -1)
            encoded_feature = layers.Dense(units=encoding_size)(encoded_feature)
        encoded_features.append(encoded_feature)
    return encoded_features


## Implement the Gated Linear Unit

[Gated Linear Units (GLUs)](https://arxiv.org/abs/1612.08083) provide the
flexibility to suppress input that are not relevant for a given task.

In [18]:

class GatedLinearUnit(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.linear = layers.Dense(units)
        self.sigmoid = layers.Dense(units, activation="sigmoid")

    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)


## Implement the Gated Residual Network

The Gated Residual Network (GRN) works as follows:

1. Applies the nonlinear ELU transformation to the inputs.
2. Applies linear transformation followed by dropout.
4. Applies GLU and adds the original inputs to the output of the GLU to perform skip
(residual) connection.
6. Applies layer normalization and produces the output.

In [19]:

class GatedResidualNetwork(layers.Layer):
    def __init__(self, units, dropout_rate):
        super().__init__()
        self.units = units
        self.elu_dense = layers.Dense(units, activation="elu")
        self.linear_dense = layers.Dense(units)
        self.dropout = layers.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = layers.LayerNormalization()
        self.project = layers.Dense(units)

    def call(self, inputs):
        x = self.elu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x


## Implement the Variable Selection Network

The Variable Selection Network (VSN) works as follows:

1. Applies a GRN to each feature individually.
2. Applies a GRN on the concatenation of all the features, followed by a softmax to
produce feature weights.
3. Produces a weighted sum of the output of the individual GRN.

Note that the output of the VSN is [batch_size, encoding_size], regardless of the
number of the input features.

In [20]:

class VariableSelection(layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = keras.ops.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = keras.ops.stack(x, axis=1)

        outputs = keras.ops.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs


## Create Gated Residual and Variable Selection Networks model

In [21]:

def create_model(encoding_size):
    inputs = create_model_inputs()
    feature_list = encode_inputs(inputs, encoding_size)
    num_features = len(feature_list)

    features = VariableSelection(num_features, encoding_size, dropout_rate)(
        feature_list
    )

    outputs = layers.Dense(units=1, activation="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


## Compile, train, and evaluate the model

In [22]:
learning_rate = 0.001
dropout_rate = 0.15
batch_size = 265
num_epochs = 20
encoding_size = 16

model = create_model(encoding_size)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
)


# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

print("Start training the model...")
train_dataset = get_dataset_from_csv(
    train_data_file, shuffle=True, batch_size=batch_size
)
valid_dataset = get_dataset_from_csv(valid_data_file, batch_size=batch_size)
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=[early_stopping],
)
print("Model training finished.")

print("Evaluating model performance...")
test_dataset = get_dataset_from_csv(test_data_file, batch_size=batch_size)
_, accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

AttributeError: module 'keras' has no attribute 'ops'

You should achieve more than 95% accuracy on the test set.

To increase the learning capacity of the model, you can try increasing the
`encoding_size` value, or stacking multiple GRN layers on top of the VSN layer.
This may require to also increase the `dropout_rate` value to avoid overfitting.

**Example available on HuggingFace**

| Trained Model | Demo |
| :--: | :--: |
| [![Generic badge](https://img.shields.io/badge/%F0%9F%A4%97%20Model-Classification%20With%20GRN%20%26%20VSN-red)](https://huggingface.co/keras-io/structured-data-classification-grn-vsn) | [![Generic badge](https://img.shields.io/badge/%F0%9F%A4%97%20Space-Classification%20With%20GRN%20%26%20VSN-red)](https://huggingface.co/spaces/keras-io/structured-data-classification-grn-vsn) |